In [1]:
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/models/yolov5


## Yolov5s Testing

In [5]:
# TENSORFLOW CONTAINER (PY37)

image_set = 'train'

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': image_set,
                            'weights': 'SM_CHANNEL_MODEL/best.pt',
                            'data': 'atrw.yaml',
                            'anno-json': 'data/atrw_detect_center.json',
                            'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test-s/{image_set}',
                            'save-s3': True
                        }
)

In [8]:
estimator.fit({'model': f's3://{bucket}/models/yolov5/train-full/',
               
               'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/Main',
               'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
               'train': f's3://{bucket}/atrw/detection/train/',
               
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

2021-05-29 03:20:50 Starting - Starting the training job...
2021-05-29 03:21:00 Starting - Launching requested ML instancesProfilerReport-1622258446: InProgress
......
2021-05-29 03:22:07 Starting - Preparing the instances for training......
2021-05-29 03:23:19 Downloading - Downloading input data.........
2021-05-29 03:24:43 Training - Downloading the training image..2021-05-29 03:25:01.685285: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-29 03:25:01.689110: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-29 03:25:01.816235: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-29 03:25:04,906 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training

2021-05-29 03:25:02 Training - Train

## Yolov5m Testing

In [10]:
# TENSORFLOW CONTAINER (PY37)

image_set = 'train'

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': image_set,
                            'weights': 'SM_CHANNEL_MODEL/best.pt',
                            'data': 'atrw.yaml',
                            'anno-json': 'data/atrw_detect_corner.json',
                            'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test-m/{image_set}',
                            'save-s3': True
                        }
)

In [12]:
estimator.fit({'model': f's3://{bucket}/models/yolov5/train-full-m/',
               
               'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/Main',
               'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
               'train': f's3://{bucket}/atrw/detection/train/',
               
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

2021-05-29 03:47:19 Starting - Starting the training job...
2021-05-29 03:47:43 Starting - Launching requested ML instancesProfilerReport-1622260034: InProgress
......
2021-05-29 03:48:43 Starting - Preparing the instances for training......
2021-05-29 03:49:43 Downloading - Downloading input data.........
2021-05-29 03:51:19 Training - Downloading the training image...
2021-05-29 03:51:44 Training - Training image download completed. Training in progress.2021-05-29 03:51:36.349475: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-29 03:51:36.353164: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-29 03:51:36.479458: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-29 03:51:38,951 sagemaker-training-toolkit INFO     Importe

## Testing Subsets

### Fractional Subsets

In [9]:
%%capture --no-stderr

sets = [16, 8, 4, 2]

for set in sets:
    estimator = TensorFlow(entry_point='test.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 1920,
                                'task': 'test',
                                'weights': 'SM_CHANNEL_MODEL/best.pt',
                                'data': f'atrw{set}.yaml',
                                'anno-json': 'data/atrw_detect_center.json',
                                'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/frac/test{set}/',
                                'save-s3': True
                            }
    )
    estimator.fit({'model': f's3://{bucket}/models/yolov5/train{set}/',
                   'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
                   'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
                   'test': f's3://{bucket}/atrw/detection/test/'})

### Clustering Split

In [9]:
%%capture --no-stderr

sets = [16, 8, 4, 2]

for set in sets:
    estimator = TensorFlow(entry_point='test.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 1920,
                                'task': 'test',
                                'weights': 'SM_CHANNEL_MODEL/best.pt',
                                'data': 'atrw.yaml',
                                'anno-json': 'data/atrw_detect_center.json',
                                'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/bg-split/test/split{set}/',
                                'save-s3': True
                            }
    )
    estimator.fit({'model': f's3://{bucket}/models/yolov5/bg-split/split{set}/',
                   'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
                   'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
                   'test': f's3://{bucket}/atrw/detection/test/'})